In [ ]:
%%capture
!pip install pingouin
!pip install researchpy
!pip install myst_nb

# Data Analysis Reporting Tool
## Resumo Geral do Dado

In [ ]:
import os
import sys
import glob
import math
import warnings
import numpy as np
import seaborn as sns
import pingouin as pg
import researchpy as rpy
from myst_nb import glue
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed
from collections import OrderedDict
from IPython.display import display

from global_configurations import *

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# COMMENT THIS LINE BEFORE BUILD
# DATASET_PATH = 'YOUR_DATASET_PATH'

DATASET_INFO_PATH = glob.glob(os.path.join(DATASET_PATH, '*.yaml'))[0]
conf_dict = read_config_file(DATASET_INFO_PATH)
df = read_dataset(conf_dict, DATASET_PATH)

df_original, numerical, categorical, categorical_dt, datetime, identifier, target = infer_column_types(df, conf_dict)

SAMPLE_SIZE = 10000
if len(df_original) > SAMPLE_SIZE:
    df = df_original.sample(SAMPLE_SIZE)
else: 
    df = df_original

### Dimensão dos dados (linhas, colunas)

In [ ]:
rows, columns = df_original[numerical + categorical + datetime + identifier + [target]].shape
glue("rows", rows)
glue("columns", columns)

### Inferir tipos das variáveis

In [ ]:
variable_types = pd.DataFrame({'Identificadores': [', '.join(identifier)],
                               'Numéricas': [', '.join(numerical)],
                               'Categóricas': [', '.join(categorical)], 
                               'Datas': [', '.join(datetime)],
                               'Alvo':[target]}).T.rename(columns={0: 'Variáveis'})

glue("variable_types", variable_types)

### Nome e tipo de dado das colunas

In [ ]:
column_types = df[numerical+categorical+categorical_dt+[target]]
column_types = column_types.dtypes.to_frame().reset_index().rename(columns={'index': 'Coluna', 0: 'Tipo'})
column_types['Tipo'] = column_types.Tipo.apply(str)
column_types = column_types.groupby('Tipo').agg({'Coluna': lambda x: ', '.join(sorted(x))})
glue("column_types", column_types)

## Resumo de Qualidade do Dado
### Dados faltantes

In [ ]:
fig = plt.figure(figsize = (18,8))
ax = sns.heatmap(df[numerical + categorical + datetime + [target]].isna(), yticklabels=1000, cbar=False)
ax.set(title='Valores nulos por coluna')

glue("missing_data_plot", fig, display=False)

In [ ]:
data = df[numerical + categorical + datetime + [target]].isna().sum()/df.shape[0]*100

fig = plt.figure(figsize = (18,8))
ax = sns.barplot(x='index', y=0, data=data.to_frame().reset_index())
ax.set(xlabel='', ylabel='%', title='% de valores nulos por coluna')
ax.tick_params(axis='x', rotation=90)
ax.axhline(data.mean(), 0, 1, color='red', label=f'média={data.mean():.2f}%')
ax.legend()

glue("percent_missing_data_plot", fig, display=False)

In [ ]:
has_na_values = ', '.join(data[data>0].index)

glue("has_na_values", has_na_values)

### Linhas duplicadas

In [ ]:
duplicated_rows = df.duplicated().sum()

glue("duplicated_rows", duplicated_rows)

## Variável Alvo

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
data = df[[target]].value_counts().reset_index()
data = data.rename(columns={0: 'value'})

sns.barplot(y='value', x=target, data=data, ax=axes[0], palette='muted')
colors = [sns.color_palette('muted')[0],
          sns.color_palette('muted')[1]]
data.plot.pie(y='value', ax=axes[1], legend=True, autopct='%1.1f%%', 
              explode=(0, 0.1), shadow=True, startangle=0, colors=colors)
axes[1].set(ylabel=target)

glue("target_plot", fig, display=False)

## Variáveis Individuais
### Numéricas

In [ ]:
numerical_stats = df[numerical].describe().T.sort_index()
numerical_stats.rename(columns={'count': 'Qtd', 
                                'mean': 'Média',
                                'std': 'Desvio Padrão', 
                                'min': 'Min', 'max': 'Max'}, inplace=True)
glue("numerical_stats", numerical_stats)

In [ ]:
fig = plt.figure(figsize = (16, 7))

for idx, var in enumerate(numerical, 1):
    ax = fig.add_subplot(2, 4, idx)
    sns.histplot(df[var], ax=ax, kde=True)
    ax.set(ylabel='')
    
fig.suptitle('Histograma das variáveis numéricas')
fig.tight_layout()

glue("numerical_hist_plot", fig, display=False)

In [ ]:
# TODO: QQPlot para variáveis numéricas

In [ ]:
fig = plt.figure(figsize = (16, 7))

for idx, var in enumerate(numerical, 1):
    ax = fig.add_subplot(2, 4, idx)
    sns.boxplot(y=var, data=df[[var]], ax=ax, palette='deep')
    ax.set(ylabel='', xlabel=var)
    
fig.suptitle('Boxplot das variáveis numéricas')
fig.tight_layout()

glue("numerical_box_plot", fig, display=False)

### Categóricas

In [ ]:
categorical_stats = df[categorical].astype('category').describe().T.sort_index()
categorical_stats.rename(columns={'count': 'Qtd', 
                                  'unique': 'Único',
                                  'top': 'Moda', 
                                  'freq': 'Frequência'}, inplace=True)

glue("categorical_stats", categorical_stats)

In [ ]:
fig = plt.figure(figsize=(10, 5))
data = df[categorical].nunique().to_frame(name='Qtd')
data = data.reset_index().sort_values(by='Qtd', ascending=False)

ax = sns.barplot(y='Qtd', x='index', data=data, palette='magma_r')
ax.set(xlabel='', title='Quantidade de valores únicos (categorias)')
ax.tick_params(axis='x', rotation=90)

glue("categorical_unique_plot", fig, display=False)

In [ ]:
fig = plt.figure(figsize=(15, 35))

for idx, var in enumerate(categorical, 1):
    ax = fig.add_subplot(10, 4, idx)
    data = df[var].value_counts()[:10]
    data = data.to_frame().reset_index()
    data.columns = ['categoria', 'qtd']
    data['qtd'] = data['qtd'].apply(lambda x: x/sum(data['qtd'])) 
    data.categoria = data.categoria.apply(lambda x: f"{x[:15]}..." if isinstance(x, str) else x)
    
    sns.barplot(x='categoria', y='qtd', data=data, ax=ax, palette=sns.color_palette("husl", 8))
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
    ax.set(xlabel="", title=var)
    
fig.suptitle('Top 10 categorias por variável', y=1)    
fig.tight_layout()

glue("categorical_top_categories_plot", fig, display=False)

### Data

In [ ]:
datetime_stats = df_original[datetime].describe().T.sort_index()
datetime_stats.rename(columns={'count': 'Qtd', 
                               'unique': 'Único',
                               'top': 'Moda', 
                               'freq': 'Frequência',
                               'first': 'Primeiro',
                               'last': 'Último'}, inplace=True)

glue("datetime_stats", datetime_stats)

In [ ]:
categorical_year = list(get_datetime_var_names(datetime, 'year'))
categorical_month = list(get_datetime_var_names(datetime, 'month'))
categorical_week_number = list(get_datetime_var_names(datetime, 'week_number'))
categorical_day = list(get_datetime_var_names(datetime, 'day'))
categorical_day_week = list(get_datetime_var_names(datetime, 'day_week'))
categorical_hour = list(get_datetime_var_names(datetime, 'hour'))
categorical_minute = list(get_datetime_var_names(datetime, 'minute'))
categorical_second = list(get_datetime_var_names(datetime, 'second'))

#### Ano

In [ ]:
fig = plt.figure(figsize=(15, 8))

for idx, var in enumerate(categorical_year, 1):
    ax = fig.add_subplot(3, 4, idx)
    data = df[var].value_counts()
    data = data.to_frame().reset_index()
    data.columns = ['categoria', 'qtd']
    data['qtd'].apply(lambda x: x/sum(data['qtd'])) 
    
    sns.barplot(x='categoria', y='qtd', data=data, ax=ax, palette='flare')
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
    ax.set(xlabel="", title=var)

fig.tight_layout()

glue("categoricaldt_year_plot", fig, display=False)

#### Mês

In [ ]:
fig = plt.figure(figsize=(15, 7))

for idx, var in enumerate(categorical_month, 1):
    ax = fig.add_subplot(3, 4, idx)
    data = df[var].value_counts()
    data = data.to_frame().reset_index()
    data.columns = ['categoria', 'qtd']
    data['qtd'].apply(lambda x: x/sum(data['qtd'])) 
    
    sns.barplot(x='categoria', y='qtd', data=data, ax=ax, palette='flare')
    ax.set(xlabel="", title=var)

fig.tight_layout()

glue("categoricaldt_month_plot", fig, display=False)

#### Semana

In [ ]:
fig = plt.figure(figsize=(15, 7))

for idx, var in enumerate(categorical_week_number, 1):
    ax = fig.add_subplot(3, 4, idx)
    data = df[var].value_counts()
    data = data.to_frame().reset_index()
    data.columns = ['categoria', 'qtd']
    data['qtd'].apply(lambda x: x/sum(data['qtd'])) 
    
    sns.barplot(x='categoria', y='qtd', data=data, ax=ax, palette='flare')
    ax.set(xlabel="", title=var)

fig.tight_layout()

glue("categoricaldt_week_plot", fig, display=False)

#### Dia

In [ ]:
fig = plt.figure(figsize=(15, 7))

for idx, var in enumerate(categorical_day, 1):
    ax = fig.add_subplot(3, 4, idx)
    data = df[var].value_counts()
    data = data.to_frame().reset_index()
    data.columns = ['categoria', 'qtd']
    data['qtd'].apply(lambda x: x/sum(data['qtd'])) 
    
    sns.barplot(x='categoria', y='qtd', data=data, ax=ax, palette='flare')
    ax.set(xlabel="", title=var)

fig.tight_layout()

glue("categoricaldt_day_plot", fig, display=False)

#### Dia da semana

In [ ]:
fig = plt.figure(figsize=(15, 7))

for idx, var in enumerate(categorical_day_week, 1):
    ax = fig.add_subplot(3, 4, idx)
    data = df[var].value_counts()
    data = data.to_frame().reset_index()
    data.columns = ['categoria', 'qtd']
    data['qtd'].apply(lambda x: x/sum(data['qtd'])) 
    
    sns.barplot(x='categoria', y='qtd', data=data, ax=ax, palette='flare')
    ax.set(xlabel="", title=var)

fig.tight_layout()

glue("categoricaldt_week_day_plot", fig, display=False)

#### Hora

In [ ]:
fig = plt.figure(figsize=(15, 7))

for idx, var in enumerate(categorical_hour, 1):
    ax = fig.add_subplot(3, 4, idx)
    data = df[var].value_counts()
    data = data.to_frame().reset_index()
    data.columns = ['categoria', 'qtd']
    data['qtd'].apply(lambda x: x/sum(data['qtd'])) 
    
    sns.barplot(x='categoria', y='qtd', data=data, ax=ax, palette='flare')
    ax.set(xlabel="", title=var)

fig.tight_layout()

glue("categoricaldt_hour_plot", fig, display=False)

#### Minuto

In [ ]:
fig = plt.figure(figsize=(15, 7))

for idx, var in enumerate(categorical_minute, 1):
    ax = fig.add_subplot(3, 4, idx)
    data = df[var].value_counts()
    data = data.to_frame().reset_index()
    data.columns = ['categoria', 'qtd']
    data['qtd'].apply(lambda x: x/sum(data['qtd'])) 
    
    sns.barplot(x='categoria', y='qtd', data=data, ax=ax, palette='flare')
    ax.set(xlabel="", title=var)

fig.tight_layout()

glue("categoricaldt_minute_plot", fig, display=False)

#### Segundo

In [ ]:
fig = plt.figure(figsize=(15, 7))

for idx, var in enumerate(categorical_second, 1):
    ax = fig.add_subplot(3, 4, idx)
    data = df[var].value_counts()
    data = data.to_frame().reset_index()
    data.columns = ['categoria', 'qtd']
    data['qtd'].apply(lambda x: x/sum(data['qtd'])) 
    
    sns.barplot(x='categoria', y='qtd', data=data, ax=ax, palette='flare')
    ax.set(xlabel="", title=var)

fig.tight_layout()

glue("categoricaldt_week_second_plot", fig, display=False)

## Classificação de Variáveis
### Top 40 variáveis numéricas com maior associação com o alvo

In [ ]:
eta_sq = {}
for v in numerical:
    aov = pg.anova(dv=v, between=target, data=df, detailed=True)
    if 'np2' not in aov.columns:
        eta_sq[v] = 0.0
        continue
    eta_sq[v] = aov['np2'][0]
eta_sq = sorted(eta_sq.items(), key=operator.itemgetter(1), reverse=True)

fig = plt.figure(figsize=(15, 5))
ax = sns.barplot(x='Variável', y='Eta-Squared', data=pd.DataFrame(eta_sq, columns=['Variável', 'Eta-Squared']), palette='magma_r')
ax.tick_params(axis='x', rotation=90)

glue("numerical_eta_sq_plot", fig, display=False)

### Top 40 variáveis categóricas com maior associação com o alvo

In [ ]:
cramers_v = {}
# TODO Identifier variables
for var in categorical:
    crosstab, res = rpy.crosstab(df[target], 
                                 df[var], test='chi-square')
    cramers_v[var] = res['results'][2]
cramers_v = sorted(cramers_v.items(), key=operator.itemgetter(1), reverse=True)

fig = plt.figure(figsize=(15, 5))
ax = sns.barplot(x='Variável', y='Cramers V', data=pd.DataFrame(cramers_v, columns=['Variável', 'Cramers V']).iloc[:40], palette='magma_r')
ax.tick_params(axis='x', rotation=90)

glue("categorical_cramers_v_plot", fig, display=False)

In [ ]:
cramers_v = {}
# TODO Identifier variables
if len(categorical_dt) > 0:
    for var in categorical_dt:
        crosstab, res = rpy.crosstab(df[target], 
                                     df[var], test='chi-square')
        cramers_v[var] = res['results'][2]
    cramers_v = sorted(cramers_v.items(), key=operator.itemgetter(1), reverse=True)

    fig = plt.figure(figsize=(15, 5))
    ax = sns.barplot(x='Variável', y='Cramers V', data=pd.DataFrame(cramers_v, columns=['Variável', 'Cramers V']).iloc[:40], palette='magma_r')
    ax.tick_params(axis='x', rotation=90)

    glue("categorical_dt_cramers_v_plot", fig, display=False)

## Relação entre variável explicavel e de resposta
### Numéricas

In [ ]:
methods = ['pearson', 'kendall', 'spearman']

fig = plt.figure(figsize=(15, 4))
for idx, m in enumerate(methods, 1):
    ax = fig.add_subplot(1, 3, idx)
    correlation_matrix(df[numerical], numerical, m, fig, ax)
    ax.tick_params(axis='x', rotation=90)
    ax.set(title=m)
fig.tight_layout()

glue("numerical_corr_matrix_plot", fig, display=False)

In [ ]:
# TODO: Explicar a relação entre as variáveis com regressão linear
def scatter_plot(data, x, y, target):
    fig, ax = plt.subplots(1)
    cols = [x] if x == y else [x, y]
    data = data[cols + [target]].fillna(0)
    return sns.scatterplot(data=data, x=x, y=y, ax=ax, hue=target)

w1, w2 = None, None

cols_list = numerical
    
w1 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[0],
    description='Variável 1:'
)
w2 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[1],
    description='Variável 2:'
)

i = interactive(scatter_plot, data=fixed(df), x=w1, y=w2, target=fixed(target))
hbox = widgets.HBox(i.children)
display(hbox)

### Categóricas

In [ ]:
fig = plt.figure(figsize=(15, 65))

def supress_category_label(lbl, size=10):
    if not isinstance(lbl, str):
        return lbl
    if len(lbl) < size:
        return lbl
    return f"{lbl[:size]}..."

for idx, var in enumerate(categorical, 1):
    ax = fig.add_subplot(14, 3, idx)
    data = df[[target, var]] 
    data[var] = data[var].apply(supress_category_label)
   
    mosaic_plot(data=data, x=target, y=var, ax=ax)

fig.tight_layout()

glue("categorical_mosaic_plot", fig, display=False)

In [ ]:
w1, w2 = None, None

if target:
    cols_list = [target] + categorical 
    cols_list = list(OrderedDict.fromkeys(cols_list)) 
else:
    cols_list = conf_dict['CategoricalColumns']
    
w1 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[0],
    description='Variável 1:'
)
w2 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[1],
    description='Variável 2:'
)

i = interactive(interactive_mosaic_plot, data=fixed(df), x=w1, y=w2)
hbox = widgets.HBox(i.children)
display(hbox)

### Numéricas e Categóricas

In [ ]:
fig = plt.figure(figsize=(12, 7))

l = len(numerical)
for idx, v in enumerate(numerical, 1):
    ax = fig.add_subplot(math.ceil(l/3), 3, idx)
    data = df[[v] + [target]].dropna()
    
    sns.boxplot(data=data, y=v, x=target, palette='magma')

fig.suptitle('Boxplot variáveis numéricas por alvo')
fig.tight_layout()

glue("numcat_boxplot", fig, display=False)

In [ ]:
pairgrid = sns.pairplot(df[numerical+ [target]].dropna(), hue=target, plot_kws={'alpha':0.5}, palette='magma')
fig = pairgrid.fig
sns.despine()

fig.suptitle('Matriz de dispersão entre variáveis numéricas separadas pelo alvo')
fig.tight_layout()

glue("numcat_scattermatrix", fig, display=False)